[[并行程序设计]]

---

## 无锁编程

**为什么要无锁编程: 互斥锁的开销**

- **锁机制**（如 `**std::mutex**` 和 `**std::lock_guard**`）通过阻塞线程来确保只有一个线程能够访问共享资源，防止数据竞争和不一致。然而，锁的使用会引入**开销**，包括：
    - 上下文切换：线程在获取不到锁时会被阻塞，操作系统需要进行上下文切换，从而增加开销。
    - CPU 资源浪费：线程在等待锁时会消耗 CPU 时间。
    - 内存同步：锁操作通常涉及内存屏障（memory barrier），确保缓存一致性，这会降低性能。
- 当多个线程需要频繁访问同一个共享资源时，锁会**串行化**访问。特别是在任务较轻且数量很多的情况下，线程池中的线程可能会花费大量时间在获取和释放锁上，而不是在实际计算上。这会导致线程池调度效率低下，程序的整体性能下降。

### 原子计数

关键区别在于对counter,有锁编程需要在修改counter前后进行锁管理,而无锁编程可以把counter作为**原子类型**,这样就不需要锁了

In [ ]:
\#include <iostream>
\#include <cstdint>
\#include <vector>
\#include <thread>
\#include <atomic> // 用于原子数据类型
\#include <mutex>  // 用于互斥锁
\#include "../include/hpc_helpers.hpp" // 假设你有一个自定义的计时器

int main() {
    // 定义互斥锁和线程向量
    std::mutex mutex;
    std::vector<std::thread> threads;

    const uint64_t num_threads = 10;
    const uint64_t num_iters = 100'000'000; // C++14 数字分隔符，表示1亿

    // 使用互斥锁的计数函数
    auto lock_count = [&] (volatile uint64_t* counter, const auto& id) -> void {
        for (uint64_t i = id; i < num_iters; i += num_threads) {
            std::lock_guard<std::mutex> lock_guard(mutex); // 加锁
            (*counter)++; // 更新计数器
        }
    };

    // 使用原子变量的计数函数
    auto atomic_count = [&] (volatile std::atomic<uint64_t>* counter, const auto& id) -> void {
        for (uint64_t i = id; i < num_iters; i += num_threads) {
            (*counter)++; // 原子操作，不需要锁
        }
    };

    // 测试使用互斥锁的多线程计数
    TIMERSTART(mutex_multithreaded);
    uint64_t counter = 0;
    threads.clear();

    for (uint64_t id = 0; id < num_threads; id++) {
        threads.emplace_back(lock_count, &counter, id);
    }
    for (auto& thread : threads) {
        thread.join(); // 等待所有线程完成
    }
    TIMERSTOP(mutex_multithreaded);

    // 测试使用原子变量的多线程计数
    TIMERSTART(atomic_multithreaded);
    std::atomic<uint64_t> atomic_counter(0);
    threads.clear();

    for (uint64_t id = 0; id < num_threads; id++) {
        threads.emplace_back(atomic_count, &atomic_counter, id);
    }
    for (auto& thread : threads) {
        thread.join(); // 等待所有线程完成
    }
    TIMERSTOP(atomic_multithreaded);

    // 输出结果
    std::cout << "Mutex Counter: " << counter << ", Atomic Counter: " << atomic_counter << std::endl;

    return 0;
}

使用atomic的编译条件: `g++ --std=c++14 -pthread -latomic`

### 非基本原子数据类型

In [ ]:
\#include <iostream>
\#include <atomic>

// 定义一个泛型的 `state_t` 结构体
template <typename x_value_t, typename y_value_t, typename z_value_t>
struct state_t {
    x_value_t x;
    y_value_t y;
    z_value_t z;
};

// `status` 模板函数，用于报告 `atomic<state_t>` 的大小和是否为无锁
template <typename R, typename S, typename T>
void status() {
    // 定义 `std::atomic` 类型的 `state_t`
    using atomic_state_t = std::atomic<state_t<R, S, T>>;

    // 输出 `atomic_state_t` 的大小和是否为无锁
    std::cout << "Size: " << sizeof(atomic_state_t)
              << "\tLock-free: " << atomic_state_t().is_lock_free()
              << std::endl;
}

int main() {
    std::cout << "Size\tLock-free?" << std::endl;

    // 检查不同类型组合的 `state_t` 是否为无锁
    status<uint8_t, uint8_t, uint16_t>();   // 32 位
    status<uint16_t, uint16_t, uint16_t>(); // 48 位
    status<uint32_t, uint32_t, uint32_t>(); // 96 位
    status<uint64_t, uint64_t, uint64_t>(); // 192 位
    status<uint32_t, uint32_t, uint8_t>();  // 混合类型

    return 0;
}

> **大小为多少的结构体,可以直接作为原子操作执行?**
> 
> - **硬件原生支持**：
>     - 一些数据类型，如 32 位（4 字节）、64 位（8 字节）和 128 位（16 字节）的数据，可以在现代硬件上直接作为**原子操作**执行，而**无需使用锁**。这意味着这些操作是线程安全的，并且效率极高，因为它们不需要等待或阻塞。
> - **非硬件支持的类型**：
>     - 对于 24 位（3 字节）、48 位（6 字节）和 80 位（10 字节）这种不常见的数据类型，硬件通常不支持直接的原子操作。因此，当你尝试将这些类型包装在 `**std::atomic**` 中时，C++ 标准库可能会**退回到基于锁的实现**，以确保线程安全。

> **程序员的选择**
> 
> - **方式一：使用基于锁的操作**：
>     - 如果你希望**节省内存**并且愿意接受一些性能损失，那么可以直接使用 `**std::atomic**`，即使它会退化为基于锁的操作。
>     - 这种方式适合在内存有限的场景下使用，因为它不会增加数据的存储大小，但会增加访问的时间开销。
> - **方式二：手动调整数据结构**：
>     - 另一种方式是**手动调整数据结构**，使其大小达到硬件支持的原子操作的对齐要求。例如，将 24 位的数据结构扩展到 32 位，将 48 位的数据结构扩展到 64 位，以此类推。
>     - 这样做的好处是，尽管增加了一些内存开销，但你可以享受到**无锁操作**带来的高性能优势，因为硬件能够直接支持这些对齐后的数据类型的原子操作

### 利用比较交换以原子方式并行化最大值归约

即使我们使用了原子操作（Atomic Operations），仍然有可能在多线程环境下导致错误的结果。这是因为**原子操作**只能确保**单一操作**是线程安全的，但无法确保多个操作之间的**一致性**。

[[举个例子解释原子变量的多个操作可能导致错误]]

使用CAS操作将原子变量的多个操作控制在原子范围:

**Compare-And-Swap**（简称 CAS）是并发编程中用于实现无锁同步的基础操作。它是一种原子操作，用于在多线程环境下安全地更新共享变量，而不需要使用锁。

> **CAS 的核心思想**  
>   
> 
> CAS 操作涉及三个值：
> 
> 1. **内存位置（address）**：要修改的变量地址,也就是原子变量的地址位置。
> 2. **预期值（expected）**：当前认为变量应该拥有的值。
> 3. **新值（desired）**：希望将变量更新为的新值。
> 
> CAS 操作的过程：
> 
> - 如果内存位置中的当前值等于 `**expected**`，则将该内存位置的值更新为 `**desired**`，操作成功。
> - 如果内存位置中的当前值不等于 `**expected**`，则操作失败，不更新任何内容。
> 
> 简单来说, 每个线程都会一致轮询原子变量是否被其他线程改变过, 如果其他线程改变了原子变量, 那么就重新轮询直到原子变量不再改变并且返回false. 如果其他线程没有改动过,那么就由该线程来执行对原子变量的操作并且返回true.
> 
> ### **C++ 中的 CAS 操作**
> 
> 在 C++11 中，`**std::atomic**` 提供了两种 CAS 方法：
> 
> - `**compare_exchange_strong()**`：强版本，确保不会发生虚假失败。
> - `**compare_exchange_weak()**`：弱版本，更高效，但可能会发生虚假失败。
> 
> ### `**compare_exchange_weak()**` **和** `**compare_exchange_strong()**` **的用法**
> 
> - `**compare_exchange_weak(expected, desired)**`：
>     - 如果 `**atomic**` 当前值等于 `**expected**`，则将其更新为 `**desired**` 并返回 `**true**`。
>     - 如果不等，则将 `**expected**` 更新为 `**atomic**` 的当前值，并返回 `**false**`。
>     - 由于虚假失败的可能性，通常在循环中使用 `**compare_exchange_weak()**`。
> 
> ### **原子操作的并行性**
> 
> - **CAS 是原子操作**：在现代处理器中，CAS 是通过硬件指令（如 x86 的 `**CMPXCHG**` 指令）实现的。该操作确保对共享内存的读取、比较和交换是不可分割的，即原子的。
> - **并行执行**：多个线程可以同时调用 CAS 操作，虽然只有一个线程会成功更新共享变量，但其他线程不会被阻塞，而是会立即重新尝试。这种行为被称为**忙等待（busy-waiting）**。
> - **无需锁**：由于 CAS 是无锁的，因此不会导致线程因为锁竞争而进入队列等待，从而避免了强制的顺序执行。

In [ ]:
\#include <iostream>
\#include <cstdint>
\#include <vector>
\#include <thread>
\#include <atomic>
\#include "../include/hpc_helpers.hpp" // 假设你有一个自定义的计时器库

int main() {
    std::vector<std::thread> threads;
    const uint64_t num_threads = 10;
    const uint64_t num_iters = 100'000'000; // C++14 数字分隔符

    // 错误的实现
    auto false_max = [&](volatile std::atomic<uint64_t>* counter, const auto& id) -> void {
        for (uint64_t i = id; i < num_iters; i += num_threads) {
            // 检查并更新最大值（非原子操作，会导致竞争条件）
            if (i > *counter) {
                *counter = i;
            }
        }
    };

    // 正确的实现，使用 CAS 循环
    auto correct_max = [&](volatile std::atomic<uint64_t>* counter, const auto& id) -> void {
        for (uint64_t i = id; i < num_iters; i += num_threads) {
            uint64_t previous = counter->load();
            // 使用 CAS 保证操作的原子性
            while (previous < i && !counter->compare_exchange_weak(previous, i)) {
                // 如果 CAS 失败，previous 会更新为最新的 counter 值
            }
        }
    };

    // 测试错误的实现
    TIMERSTART(incorrect_max);
    std::atomic<uint64_t> false_counter(0);
    threads.clear();
    for (uint64_t id = 0; id < num_threads; id++) {
        threads.emplace_back(false_max, &false_counter, id);
    }
    for (auto& thread : threads) {
        thread.join();
    }
    TIMERSTOP(incorrect_max);

    // 测试正确的实现
    TIMERSTART(correct_max);
    std::atomic<uint64_t> correct_counter(0);
    threads.clear();
    for (uint64_t id = 0; id < num_threads; id++) {
        threads.emplace_back(correct_max, &correct_counter, id);
    }
    for (auto& thread : threads) {
        thread.join();
    }
    TIMERSTOP(correct_max);

    // 输出结果
    std::cout << "False max: " << false_counter.load() << "\n"
              << "Correct max: " << correct_counter.load() << std::endl;

    return 0;
}

### 任意原子操作

In [ ]:
\#include <iostream>
\#include <cstdint>
\#include <vector>
\#include <thread>
\#include <atomic>
\#include <functional>
\#include "../include/hpc_helpers.hpp" // 假设你有自定义的计时器库

// 二元原子操作函数模板
template <
		typename atomic_t, 
		typename value_t, 
		typename func_t, 
		typename pred_t>
value_t binary_atomic(
		atomic_t& atomic, 
		const value_t& operand, 
		func_t function, 
		pred_t predicate) {
		
    value_t expect = atomic.load();
    value_t target;
    
    do {
        // 计算新的值
        target = function(expect, operand);

        // 如果不满足谓词条件，则立即返回
        if (!predicate(target)) {
            return expect;
        }
    // 尝试以原子方式交换新旧值
    } while (!atomic.compare_exchange_weak(expect, target));

    // 返回更新成功后的新值或未成功交换时的旧值
    return expect;
}

int main() {
    std::vector<std::thread> threads;
    const uint64_t num_threads = 10;
    const uint64_t num_iters = 1'000'000'000; // 1 billion iterations

    // 定义线程任务：原子地更新最大偶数值
    auto even_max = [&](std::atomic<uint64_t>* counter, const auto& id) -> void {
        // 定义最大值计算函数
        auto func = [](const auto& lhs, const auto& rhs) {
            return lhs > rhs ? lhs : rhs;
        };
        // 定义谓词函数：检查是否为偶数
        auto pred = [](const auto& val) {
            return val % 2 == 0;
        };

        // 使用二元原子操作更新最大偶数值
        for (uint64_t i = id; i < num_iters; i += num_threads) {
            binary_atomic(*counter, i, func, pred);
        }
    };

    TIMERSTART(even_max);
    std::atomic<uint64_t> even_counter(0);

    // 创建并启动多个线程
    for (uint64_t id = 0; id < num_threads; id++) {
        threads.emplace_back(even_max, &even_counter, id);
    }

    // 等待所有线程完成
    for (auto& thread : threads) {
        thread.join();
    }
    TIMERSTOP(even_max);

    // 输出最终的最大偶数值
    std::cout << "Final even max: " << even_counter.load() << std::endl;

    return 0;
}

通过predicate来实现复杂的判断

In [ ]:
\#include <iostream>
\#include <cstdint>
\#include <vector>
\#include <thread>
\#include <atomic>
\#include "./include/hpc_helpers.hpp" // 修正 include 文件路径中的符号错误

// 泛型模板函数 ternary_atomic
template <
    typename atomic_t,
    typename value_t,
    typename funcp_t,
    typename funcn_t,
    typename predc_t>
value_t ternary_atomic(
    atomic_t& atomic,
    const value_t& operand,
    funcp_t pos_function,
    funcn_t neg_function,
    predc_t predicate) {
    
    value_t expect = atomic.load();
    value_t target;

    do {
        // 根据谓词选择操作
        if (predicate(expect, operand))
            target = pos_function(expect, operand);
        else
            target = neg_function(expect, operand);

        // 尝试原子交换
    } while (!atomic.compare_exchange_weak(expect, target));

    // 返回最终值（无论交换是否成功）
    return expect;
}

int main() {
    // 定义线程列表
    std::vector<std::thread> threads;
    const uint64_t num_threads = 10;    // 修正拼写错误
    const uint64_t num_iters = 100'000'000; // 修正拼写错误

    auto even_max = [&] (volatile std::atomic<uint64_t>* counter, const auto& id) -> void {
        // 定义正操作函数
        auto pos_func = [] (const auto& lhs, const auto& rhs) {
            return lhs;
        };

        // 定义负操作函数
        auto neg_func = [] (const auto& lhs, const auto& rhs) {
            return rhs;
        };

        // 定义谓词函数
        auto pred = [] (const auto& lhs, const auto& rhs) {
            return lhs > rhs && lhs % 2 == 0;
        };

        for (uint64_t i = id; i < num_iters; i += num_threads)
            ternary_atomic(*counter, i, pos_func, neg_func, pred);
    };

    TIMERSTART(even_max) // 假设 TIMERSTART 是某种计时宏定义
    std::atomic<uint64_t> even_counter(0);

    // 启动多个线程
    for (uint64_t id = 0; id < num_threads; id++)
        threads.emplace_back(even_max, &even_counter, id);

    for (auto& thread : threads)
        thread.join();

    TIMERSTOP(even_max) // 假设 TIMERSTOP 是某种计时宏定义

    // 输出结果
    std::cout << "Final Counter: " << even_counter << std::endl;

    return 0;
}

### ABA问题

**ABA 问题**是并发编程中一个常见的陷阱，特别是在使用无锁同步机制（如 CAS，即 Compare-And-Swap）时。

**ABA问题产生的原因:**

当一个线程通过 CAS 操作检查变量的值是否等于预期值时，可能会发生以下情况：

1. 变量的值从 **A** 变成 **B**，然后又变回 **A**。
2. CAS 操作会认为变量没有发生变化（因为当前值还是 **A**），从而成功完成操作。
3. 但实际上，变量在此期间经历了其他线程的修改（从 A -> B -> A），导致程序逻辑可能出现错误。

**ABA 问题的本质是：**变量的值虽然恢复到原值，但其语义状态可能已经发生了变化。

**解决 ABA 问题的方法**

**引入版本号（带标记的指针）:**通过将变量与一个**版本号**绑定在一起，可以检测变量是否发生了修改，即使其值未变。常见的实现方式是将变量和版本号组合成一个整体。

## 工作共享线程池

### 示例:串行二叉树遍历

In [ ]:
\#include <iostream>
\#include <cstdint>
\#include "../include/hpc_helpers.hpp" // 假设你有自定义计时器

// 模拟消耗计算周期的函数
void waste_cycles(uint64_t num_cycles) {
    volatile uint64_t counter = 0; // 防止编译器优化
    for (uint64_t i = 0; i < num_cycles; i++) {
        counter++;
    }
}

// 递归遍历二叉树
void traverse(uint64_t node, uint64_t num_nodes) {
    if (node < num_nodes) {
        // 模拟在当前节点进行一些工作
        waste_cycles(1 << 15); // 消耗计算周期
        
        // 遍历左子节点
        traverse(2 * node + 1, num_nodes);
        // 遍历右子节点
        traverse(2 * node + 2, num_nodes);
    }
}

int main() {
    const uint64_t num_nodes = 1 << 20; // 二叉树节点数
    
    // 计时二叉树遍历的执行时间
    TIMERSTART(traverse);
    traverse(0, num_nodes); // 从根节点开始遍历
    TIMERSTOP(traverse);

    return 0;
}

> **为什么** `**volatile**` **阻止优化？**
> 
> 1. **编译器的优化目标**：
>     - 如果编译器发现 `**counter**` 是局部变量，并且它的值对程序的外部行为没有影响，它可能会直接将循环替换为：这种优化避免了不必要的循环执行，但这违背了 `**waste_cycles**` 函数的初衷（即模拟计算工作负载）。
>         
>         ```C++
>         counter += num_cycles;
>         ```
>         
> 2. `**volatile**` **的作用**：
>     - `**volatile**` 告诉编译器：**这个变量可能会被外部因素（如硬件或多线程）修改**，因此，编译器不得对其进行优化。

In [ ]:
\#include <iostream>
\#include <cstdint>
\#include "threadpool.hpp"            // 假设你有一个线程池实现
\#include "../include/hpc_helpers.hpp" // 假设你有计时器工具

Threadpool TP(8); // 创建线程池，包含 8 个线程

// 模拟消耗计算周期的函数
void waste_cycles(uint64_t num_cycles) {
    volatile uint64_t counter = 0; // 使用 volatile 防止编译器优化
    for (uint64_t i = 0; i < num_cycles; i++) {
        counter++;
    }
}

// 遍历二叉树
void traverse(uint64_t node, uint64_t num_nodes) {
    if (node < num_nodes) {
        // 在当前节点执行一些工作
        waste_cycles(1 << 15);

        // 使用线程池执行左子树遍历
        TP.spawn(traverse, 2 * node + 1, num_nodes);

        // 顺序执行右子树遍历
        traverse(2 * node + 2, num_nodes);
    }
}

int main() {
    const uint64_t num_nodes = 1 << 20; // 二叉树的总节点数

    // 开始计时
    TIMERSTART(traverse);

    // 使用线程池从根节点开始遍历二叉树
    TP.spawn(traverse, 0, num_nodes);

    // 等待所有任务完成并停止线程池
    TP.wait_and_stop();

    // 停止计时
    TIMERSTOP(traverse);

    return 0;
}

### 实现工作共享

与上一章全部用多线程不同,在本例使用的二叉树遍历中, 只将部分内容做了多线程处理, 主线程上仍然在执行任务.因此我们相较于上一章中的线程池我们要增加的方法为:

1. spawn方法,将部分任务委派给空闲线程
2. 引入新的条件变量cv_wait,保证主线程在最后结束,否则其他分支线程也会提前结束导致错误.
3. 在线程结束任务后, 要判断是否还有分支线程在工作, 如果没有线程在工作了就用cv_wait通知主线程

In [ ]:
\#ifndef THREADPOOL_HPP
\#define THREADPOOL_HPP

\#include <cstdint>
\#include <future>
\#include <vector>
\#include <queue>
\#include <thread>
\#include <functional>
\#include <mutex>
\#include <condition_variable>

// 线程池类定义
class ThreadPool {
private:
    // 存储线程和任务
    std::vector<std::thread> threads; // 工作线程
    std::queue<std::function<void()>> tasks; // 任务队列

    // 用于线程间同步的原语
    std::mutex mutex; // 互斥锁
    std::condition_variable cv,cv_wait; // 条件变量

    // 线程池的状态
    bool stop_pool; // 指示线程池是否停止
    uint32_t active_threads; // 当前活动线程数量
    const uint32_t capacity; // 线程池的容量

    template <
        typename Func,
        typename... Args,
        typename Rtrn = typename std::result_of<Func(Args...)>::type>
    auto make_task(Func&& func, Args&&... args) -> std::packaged_task<Rtrn(void)> {
        auto aux = std::bind(std::forward<Func>(func), std::forward<Args>(args)...);
        return std::packaged_task<Rtrn(void)>(aux);
    }

    // 任务执行前的钩子
    void before_task_hook() {
        active_threads++; 
    }

    // 任务执行后的钩子
    void after_task_hook() {
        active_threads--; // 活动线程计数减少

        // 如果没有活动线程且任务队列为空
        if (active_threads == 0 && tasks.empty()) {
            stop_pool = true; // 停止线程池
            cv_wait.notify_one();  // 通知等待中的线程
        }
    }

    public:
    ThreadPool(uint64_t capacity)
        : stop_pool(false), active_threads(0), capacity(capacity) 
    {
        // 工作线程的循环函数
        auto wait_loop = [this]() -> void {
            while (true) {
                // 占位符任务
                std::function<void(void)> task;
                
                // 进入锁定区域，等待任务或停止信号
                {
                    std::unique_lock<std::mutex> unique_lock(mutex);
                    auto predicate = [this]() -> bool {
                        // 唤醒条件：线程池停止或任务队列不为空
                        return stop_pool || !tasks.empty();
                    };

                    // 等待唤醒，直到满足唤醒条件
                    cv.wait(unique_lock, predicate);

                    // 如果线程池停止且没有任务可执行，退出线程
                    if (stop_pool && tasks.empty()) return;

                    // 从任务队列中取出一个任务
                    task = std::move(tasks.front());
                    tasks.pop();

                    // 任务计数增加
                    before_task_hook();
                } // 离开锁定区域，允许其他线程访问任务队列

                // 执行任务
                task();

                // 任务完成后，调整活动线程计数
                {
                    std::lock_guard<std::mutex> lock_guard(mutex);
                    after_task_hook();
                }
            }
        };
        // 创建并启动指定数量的线程
        for (uint64_t id = 0; id < capacity; id++) {
            threads.emplace_back(wait_loop);
        }
    }     

    ~ThreadPool() {
        {
            // 加锁以确保线程池的安全访问
            std::lock_guard<std::mutex> lock_guard(mutex);
            // 修改线程池状态以停止所有线程
            stop_pool = true;
        } // 这里释放锁

        // 通知所有等待的线程停止
        cv.notify_all();

        // 等待所有线程完成工作并退出
        for (auto& thread : threads) {
            if (thread.joinable()) {
                thread.join(); // 确保线程安全退出
            }
        }
    }

    template <typename Func, typename... Args, typename Rtrn = typename std::result_of<Func(Args...)>::type>
    auto enqueue(Func&& func, Args&&... args) -> std::future<Rtrn> {
        // 创建任务并获取任务的 future
        auto task = make_task(func,args...);
        auto future = task.get_future();
        
        // 将任务封装到 shared_ptr 中，便于复制到任务队列
        auto task_ptr = std::make_shared<std::packaged_task<Rtrn()>>(std::move(task));

        {
            // 加锁以确保线程安全
            std::lock_guard<std::mutex> lock_guard(mutex);

            // 如果线程池已经停止，禁止添加新任务
            if (stop_pool) {
                throw std::runtime_error("enqueue on stopped ThreadPool");
            }

            // 将任务包装成 void 函数对象，便于存储到任务队列中
            auto payload = [task_ptr]() -> void {
                (*task_ptr)(); // 调用任务
            };

            // 将任务添加到任务队列
            tasks.emplace(payload);
        }

        // 通知一个线程去处理任务
        cv.notify_one();

        // 返回 future 以便获取任务结果
        return future;
    }

    // Public member function to wait and stop the thread pool
    void wait_and_stop() {
        // Acquire a unique lock for the mutex
        std::unique_lock<std::mutex> unique_lock(mutex);

        // Define the predicate to check when the pool should stop
        auto predicate = [&]() -> bool {
            return stop_pool;
        };

        // Wait until the predicate is satisfied
        std::cout << "[ThreadPool] Waiting for thread pool to stop..." << std::endl;
        cv_wait.wait(unique_lock, predicate);
        std::cout << "[ThreadPool] Thread pool has stopped." << std::endl;
    }
    
    template <
        typename Func, 
        typename... Args>
    void spawn(
        Func&& func, 
        Args&&... args) {

        if (active_threads < capacity) 
            // 如果有空闲线程，将任务加入线程池
            enqueue(func,args...);
        else
            // 如果线程池已满，则直接在当前线程顺序执行任务
            func(args...);
    }
};

\#endif

## 并行图搜索

### 二元背包问题


二元背包问题（0/1 Knapsack Problem）是一个经典的组合优化问题，其目标是：

- **给定**：
    - 一组物品，每个物品有一个 **价值** _vi_ 和 **重量** _wi_
    - 一个背包，具有最大容量 _C_
- **目标**：选择物品的子集，使得这些物品的总价值最大，且总重量不超过背包的容量 _C_


**分支定界法**

为了更高效地探索解空间，分支定界法通过剪枝减少了不必要的计算：

- 将状态空间建模为 **二叉决策树**。
- **每一层**的节点对应一个物品，每个节点分为两种可能：
    1. **选择当前物品**（向左子树前进）。
    2. **跳过当前物品**（向右子树前进）。
- 二叉树的 **深度** 等于物品的数量 _n_，总节点数最多为 $2^n$。


**状态空间表示**

**节点状态**：

- 当前选择的物品集合，用一个位掩码（`**bitmask**`）表示，长度为 _n_。
- 当前累计的 **价值** 和 **重量**。

**全局状态**：

- 当前已知的 **最优解值**（`**global-state**`）。

**计算上界和剪枝**

**累计重量检查**：

- 如果某个节点的累计重量超过容量 _C_，则 **剪枝** 当前节点及其子树。

**局部上界计算**（基于乐观估计）：

- 假设当前物品按照价值密度（价值 / 重量）排序，在不考虑容量限制的情况下，继续选择尽可能多的物品。
- 这个 **上界值** 表示当前节点及其子树可能达到的最大价值。如果这个上界小于全局最优解值，则可以剪枝。

**更新全局最优解**：

- 如果当前节点状态是一个有效解且其累计价值高于全局最优解值，则更新全局最优解值。

### 串行执行

In [ ]:
\#include <algorithm>    // std::sort
\#include <iostream>     // std::cout
\#include <vector>       // std::vector
\#include <random>       // std::uniform_int_distribution

// 定义物品的基本结构，包括价值和重量
template <typename value_t_, typename weight_t_>
struct generic_tuple_t {
    value_t_ value;      // 物品的价值
    weight_t_ weight;    // 物品的重量

    // 方便访问的类型别名
    typedef value_t_ value_t;
    typedef weight_t_ weight_t;

    // 构造函数
    generic_tuple_t(value_t_ value_, weight_t_ weight_)
        : value(value_), weight(weight_) {}
};

// 定义背包状态，包括选择的物品掩码和当前总价值
template <typename bmask_t_, typename value_t_>
struct state_t {
    bmask_t_ bmask = 0;   // 选择的物品掩码（bitmask）
    value_t_ value = 0;   // 当前背包中的总价值

    // 类型别名，便于访问
    typedef bmask_t_ bmask_t;
    typedef value_t_ value_t;
};

// 定义类型别名，方便使用
typedef uint64_t index_t;             // 索引类型
typedef uint32_t bmask_t;             // 掩码类型
typedef uint32_t value_t;             // 价值类型
typedef uint32_t weight_t;            // 重量类型

typedef generic_tuple_t<value_t, weight_t> tuple_t; // 物品元组类型

// 全局变量
state_t<bmask_t, value_t> global_state;    // 背包的全局状态
const value_t capacity = 1500;            // 背包的总容量
const index_t num_items = 32;             // 物品的总数
std::vector<tuple_t> tuples;              // 存储物品的容器

In [ ]:
template <typename tuple_t, typename index_t>
void init_tuples(std::vector<tuple_t>& tuples, index_t num_entries, 
                 int min_value = 80, int max_value = 100,
                 int min_weight = 80, int max_weight = 100) {
    // 恢复 tuple_t 的类型
    typedef typename tuple_t::value_t value_t;
    typedef typename tuple_t::weight_t weight_t;

    // 随机数生成器
    std::random_device rd;
    std::mt19937 engine(rd()); // 使用随机设备生成种子
    std::uniform_int_distribution<value_t> rho_v(min_value, max_value);
    std::uniform_int_distribution<weight_t> rho_w(min_weight, max_weight);

    // 生成物品列表
    for (index_t index = 0; index < num_entries; index++) {
        tuples.emplace_back(rho_v(engine), rho_w(engine));
    }

    // 按照价值密度排序
    auto predicate = [] (const auto& lhs, const auto& rhs) -> bool {
        return lhs.value * rhs.weight > rhs.value * lhs.weight;
    };
    std::sort(tuples.begin(), tuples.end(), predicate);
}

5.14解空间的递归遍历:

In [ ]:
template <typename tuple_t, typename bmask_t>
void sequential_update(tuple_t tuple, bmask_t bmask) {
    if (global_state.value < tuple.value) {
        global_state.value = tuple.value; // 更新全局最优解的价值
        global_state.bmask = bmask;      // 更新对应的二进制掩码
    }
}

> **功能：**
> 
> - 检查当前解的价值是否大于全局最优解。
> - 如果更优，则更新全局状态，记录新的最优值和选择的物品组合。

In [ ]:
template <typename index_t, typename tuple_t>
typename tuple_t::value_t dantzig_bound(index_t height, tuple_t tuple) {
    auto predicate = [&](const index_t& i) {
        return i < num_items && tuple.weight < capacity;
    };

    // 贪心地加入物品直到超出背包容量
    for (index_t i = height; predicate(i); i++) {
        tuple.value += tuples[i].value;
        tuple.weight += tuples[i].weight;
    }

    return tuple.value;
}

> **功能：**
> 
> - 从当前节点开始（`**height**`），按照物品的价值密度顺序贪心地选取物品，直到背包满。
> - 返回该节点的局部上界（可能的最大价值）。
> 
> **作用：**
> 
> - 用于剪枝。如果局部上界小于当前全局最优解（`**global_state.value**`），则可以剪掉该分支。

In [ ]:
template <typename index_t, typename tuple_t, typename bmask_t>
void traverse(index_t height, tuple_t tuple, bmask_t bmask) {
    // 检查当前层是否选择物品
    const bool bit = (bmask >> height) % 2;
    tuple.weight += bit * tuples[height].weight; // 更新重量
    tuple.value += bit * tuples[height].value;  // 更新价值

    // 剪枝条件 1：当前重量超过背包容量
    if (tuple.weight > capacity)
        return;

    // 更新全局状态（lower bound）
    sequential_update(tuple, bmask);

    // 剪枝条件 2：局部上界小于全局下界
    auto bsf = global_state.value; // 当前全局最优解
    if (dantzig_bound(height + 1, tuple) < bsf)
        return;

    // 继续递归生成候选解
    if (height + 1 < num_items) {
        traverse(height + 1, tuple, bmask + (1 << (height + 1))); // 左分支
        traverse(height + 1, tuple, bmask);                       // 右分支
    }
}

> **整体逻辑**
> 
> 1. **状态空间表示**：
>     - 状态空间以二叉树形式表示，每一层对应一个物品。
>     - 每个节点表示当前选取物品的状态（选择或跳过）。
> 2. **剪枝条件**：
>     - 当前重量超过背包容量时，剪掉分支。
>     - 局部上界小于全局下界时，剪掉分支。
> 3. **递归过程**：
>     - 通过递归遍历左子树和右子树，分别表示选择当前物品或跳过当前物品。
>     - 更新全局最优解（`**global_state**`）并通过 `**dantzig_bound**` 剪枝。
> 4. **贪心估计（Dantzig 上界）**：
>     - 使用贪心算法快速估计局部上界，帮助快速剪枝无效的分支。

In [ ]:
int main() {
    // 初始化物品列表
    init_tuples<tuple_t, index_t>(tuples, num_items);

    // 开始递归遍历二叉树的左右分支
    traverse(0, tuple_t(0, 0), 0); // 从根节点开始，掩码为 0
    traverse(0, tuple_t(0, 0), 1); // 从根节点开始，掩码为 1

    // 输出最终解的价值
    std::cout << "Final Optimal Value: " << global_state.value << std::endl;

    // 解码二进制掩码以显示选中物品
    auto bmask = global_state.bmask;
    std::cout << "Selected Items: ";
    for (index_t i = 0; i < num_items; i++) {
        std::cout << (bmask % 2) << " "; // 输出当前物品是否被选中
        bmask >>= 1;                     // 将掩码右移一位
    }
    std::cout << std::endl;

    return 0;
}

### 并行执行

### 并行化可能的挑战

### 线程调度开销

- **问题**：
    - 在递归过程中，分支非常轻量且频繁。如果为每个分支启动一个线程，线程池的调度开销会显著增加，甚至可能抵消并行化带来的性能提升。
- **解决方案**：
    - **分层并行化**：
        - 例如，对二叉树的顶层节点进行并行分配，每个线程独立处理一个较大的子树。
        - 随着深度增加，节点的任务变得更小，可以逐渐切换到单线程递归。
    - **线程池优化**：
        - 减少线程池任务切换和调度的频率，确保任务粒度足够大（例如，每个任务包含多个分支节点）。

### 竞争条件

- **问题**：
    - 由于全局状态 `**global_state**` 是共享资源，多个线程可能同时更新它，导致竞争条件。
- **解决方案**：
    - **使用原子操作**：
        - 用 `**std::atomic**` 包装 `**global_state**`，确保全局状态的更新是线程安全的。
        - 原子操作可以通过 CAS（Compare-And-Swap）实现锁的免锁更新。

### 任务分配与负载均衡

- **问题**：
    - 二叉树中不同分支的计算量可能不均衡（例如，一些分支很快被剪枝，而另一些需要深入递归）。
- **解决方案**：
    - **工作共享（Work Stealing）**：
        - 如果一个线程空闲，可以从其他线程的任务队列中偷取工作。
        - 使用双端队列（Deque）实现任务队列，每个线程有自己的任务队列，其他线程可以从队列末尾偷取任务。
    - **动态调度**：
        - 采用动态任务分配策略，例如按需分配任务而不是静态划分二叉树的分支。


**原子更新全局最优解:**

In [ ]:
std::atomic<state_t<bmask_t, value_t>> global_state = {{0, 0}};

In [ ]:
template <typename tuple_t, typename bmask_t>
void atomic_update(tuple_t tuple, bmask_t bmask) {
    // 恢复 tuple_t 的 value_t 类型
    typedef typename tuple_t::value_t value_t;

    // 加载当前全局状态（global_state）
    auto g_state = global_state.load();
    auto l_value = tuple.value; // 当前候选解的价值
    state_t<bmask_t, value_t> target;

    do {
        // 如果当前全局状态的价值已经更优，则无需更新
        if (g_state.value > l_value)
            return;

        // 构造目标状态
        target.value = l_value;   // 更新候选解的价值
        target.bmask = bmask;    // 更新候选解的二进制掩码
    } while (!global_state.compare_exchange_weak(g_state, target));
}

**主函数更新:**

In [ ]:
int main() {
    // 创建线程池，包含 2 个线程
    ThreadPool TP(2);

    // 初始化物品
    init_tuples<tuple_t, index_t>(tuples, num_items);

    // 将左右分支分配给线程池
    TP.spawn(traverse<index_t, tuple_t, bmask_t>, 0, tuple_t(0, 0), 0); // 左分支
    TP.spawn(traverse<index_t, tuple_t, bmask_t>, 0, tuple_t(0, 0), 1); // 右分支

    // 等待所有任务完成
    TP.wait_and_stop();

    // 打印最终结果
    auto g_state = global_state.load(); // 加载全局最优解
    std::cout << "Final Value: " << g_state.value << std::endl;

    // 打印选中物品的二进制掩码
    auto bmask = g_state.bmask;
    std::cout << "Selected Items: ";
    for (index_t i = 0; i < num_items; i++) {
        std::cout << (bmask % 2) << " ";
        bmask >>= 1;
    }
    std::cout << std::endl;

    return 0;
}